In [ ]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
from google.colab import drive
from PIL import Image
import matplotlib.pyplot as plt
import sys
import pandas as pd
from collections import defaultdict
import json
import random
from torch.utils.data import Dataset
import numpy as np
import torchvision.models.detection as detection
from torch.utils.data import DataLoader
import torch.utils
import torch
from PIL import Image as read_image
from torchvision.io import read_image
from torchvision.transforms import v2
from transforms import RandomPhotometricDistort, RandomZoomOut, RandomIoUCrop, RandomHorizontalFlip,RandomVerticalFlip ,ToTensor, Compose
from torchvision.transforms.functional import convert_image_dtype

In [ ]:
drive.mount('/content/drive')
path_base = '/content/drive/MyDrive/Academico/TD8 personal '
sys.path.append(path_base)

Mounted at /content/drive


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:

train_file = path_base + '/classificationDownload (1)/train.csv'
test_file = path_base + '/classificationDownload (1)/test.csv'
classification_file = path_base + '/classificationDownload (1)/classifications.csv'

train_images = pd.read_csv(train_file)['image_filename'].tolist()
test_images = pd.read_csv(test_file)['image_filename'].tolist()

classification_data = pd.read_csv(classification_file)

train_data = classification_data[classification_data['image_filename'].isin(train_images)]
test_data = classification_data[classification_data['image_filename'].isin(test_images)]

def get_transform(train=True):
    transforms = []
    if train:
        transforms.append(RandomPhotometricDistort())
        transforms.append(RandomZoomOut(fill=list((207., 202., 205.))))
        transforms.append(RandomHorizontalFlip(p=0.5))
        transforms.append(RandomVerticalFlip(p=0.5))

    transforms.append(v2.ToDtype(torch.float, scale=True))
    return Compose(transforms)

def group_detections(data):
    grouped_data = {}
    for _, row in data.iterrows():
        image_name = row['image_filename']
        if image_name not in grouped_data:
            grouped_data[image_name] = {
                'image_filename': image_name,
                'image_id': row['image_id'],
                'image_doi': row['image_doi'],
                'classifications': []
            }

        grouped_data[image_name]['classifications'].append({
            'nucleus_x': row['nucleus_x'],
            'nucleus_y': row['nucleus_y'],
            'bethesda_system': row['bethesda_system'],
            'cell_id': row['cell_id']
        })
    return list(grouped_data.values())


train_data_grouped = group_detections(train_data)
test_data_grouped = group_detections(test_data)


class PAPDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        self.label_map = {
            "SCC": 1,
            "HSIL": 2,
            "LSIL": 3,
            "ASC-H": 2,
            "ASC-US": 3,
            "Negative for intraepithelial lesion": 4
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = read_image(path_base + '/Imagenes/' + item['image_filename'])

        image_doi = item['image_doi']
        image_id = item['image_id']
        boxes = []
        labels = []
        for classification in item['classifications']:
            nucleus_x = classification['nucleus_x']
            nucleus_y = classification['nucleus_y']
            x_min = nucleus_x - 50
            y_min = nucleus_y - 50
            x_max = nucleus_x + 50
            y_max = nucleus_y + 50
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(self.label_map[classification['bethesda_system']])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        target = {
            "boxes": boxes,
            "labels": labels,
            "area": area,
            "image_id": image_id
        }

        num_objs = boxes.shape[0]
        target["iscrowd"] = torch.zeros((num_objs,), dtype=torch.int64)

        if self.transform:
            img, target = self.transform(img, target)
        return img, target



train_dataset = PAPDataset(train_data_grouped, transform=get_transform(train=True))
test_dataset = PAPDataset(test_data_grouped, transform=get_transform(train=False))



In [ ]:
from ssd import SSDFeatureExtractorResNet_Upgraded, ssd512_resnet50_Upgraded
model = ssd512_resnet50_Upgraded(pretrained = False, load = False)
model = model.to(device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 132MB/s]


In [ ]:
from ssd import SSDClassificationHead, SSDRegressionHead
num_classes = 4+1  # Ejemplo con 4 clases + 1 clase background


in_channels = [8192, 3584, 3584, 3328, 256, 256]
num_anchors = [4, 6, 6, 6, 4, 4]
model.head.classification_head = SSDClassificationHead(in_channels, num_anchors, num_classes)


In [ ]:

def collate_fn(batch):
    return tuple(zip(*batch))



batch_size = 16

train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_batch_sampler = train_batch_sampler = torch.utils.data.BatchSampler(
            train_sampler, batch_size, drop_last=True)


data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_sampler=train_batch_sampler,
        collate_fn=collate_fn)


data_loader_test = torch.utils.data.DataLoader(
        test_dataset, batch_size=1,
        sampler=test_sampler,
        collate_fn=collate_fn)

In [ ]:
import math
from utils import MetricLogger, warmup_lr_scheduler, SmoothedValue
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:

        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if not isinstance(v, int) else v for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        print(loss_value)

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            break

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger


'\nMetricLogger te ayuda a monitorear métricas a lo largo del entrenamiento.\nWarmup Learning Rate Scheduler te permite empezar el entrenamiento de manera más suave ajustando gradualmente la tasa de aprendizaje durante las primeras iteraciones para evitar inestabilidad.\n'

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py
!pip install -U pycocotools

--2024-11-22 11:03:41--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4063 (4.0K) [text/plain]
Saving to: ‘engine.py’

engine.py           100%[===================>]   3.97K  --.-KB/s    in 0s      

2024-11-22 11:03:41 (85.4 MB/s) - ‘engine.py’ saved [4063/4063]

--2024-11-22 11:03:41--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8388 (8.2K) [text/plain]
S

In [ ]:
from torchvision.datasets import VOCDetection
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, Normalize
from engine import evaluate
import time
import datetime
import copy


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
        params, lr = 0.003, momentum=0.9, weight_decay = 0.0005)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,110], gamma=0.1)
model.to(device)
best_ap = 0.0

print("Start training")
start_time = time.time()
for epoch in range(50):
  train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
  lr_scheduler.step()
  ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
  if ap > best_ap:
    best_ap = ap
    best_model_weights = copy.deepcopy(model.state_dict())

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))


Start training
27.227962493896484
Epoch: [0]  [ 0/22]  eta: 0:04:05  lr: 0.000146  time: 11.1724  data: 7.8212  max mem: 11604
27.734466552734375
19.733976364135742
17.919626235961914
14.095425605773926
13.748941421508789
15.368843078613281
11.137033462524414
9.613279342651367
9.138290405273438
9.148984909057617
9.218214988708496
9.462841987609863
8.557695388793945
9.240252494812012
7.855988502502441
8.27641487121582
6.990052700042725
7.029696941375732
8.120697021484375
6.733822822570801
Epoch: [0]  [20/22]  eta: 0:00:14  lr: 0.003000  time: 6.9825  data: 6.6165  max mem: 12573
8.01987075805664
Epoch: [0]  [21/22]  eta: 0:00:07  lr: 0.003000  time: 6.9435  data: 6.5795  max mem: 12573
Epoch: [0] Total time: 0:02:37 (7.1630 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:14  model_time: 0.2714 (0.2714)  evaluator_time: 0.0275 (0.0275)  time: 0.3643  data: 0.0616  max mem: 12573
Test:  [39/40]  eta: 0:00:00  model_time: 0.0157 (0.0223)  evaluator_time: 0.0162 (0.0373) 

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded_heuristica2_last.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.3144438550401475


In [ ]:
start_time = time.time()


for epoch in range(50, 100):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = copy.deepcopy(model.state_dict())

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.7662242650985718
Epoch: [50]  [ 0/22]  eta: 0:00:50  lr: 0.003000  time: 2.3025  data: 1.9890  max mem: 13132
2.0643091201782227
2.271559238433838
2.3409035205841064
2.870553970336914
2.1460750102996826
2.090892791748047
2.306033134460449
1.876212477684021
1.7936428785324097
2.086578845977783
2.2319161891937256
2.342007637023926
2.4829516410827637
2.639090061187744
1.9446046352386475
2.176784038543701
2.5954365730285645
1.9530974626541138
2.0106544494628906
1.9849692583084106
Epoch: [50]  [20/22]  eta: 0:00:04  lr: 0.003000  time: 2.2526  data: 1.9029  max mem: 13132
1.9209908246994019
Epoch: [50]  [21/22]  eta: 0:00:02  lr: 0.003000  time: 2.2168  data: 1.8642  max mem: 13132
Epoch: [50] Total time: 0:00:49 (2.2565 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:04  model_time: 0.0164 (0.0164)  evaluator_time: 0.0187 (0.0187)  time: 0.1006  data: 0.0617  max mem: 13132
Test:  [39/40]  eta: 0:00:00  model_time: 0.0158 (0.0160)  evaluator_time: 0.0180 (0.0296)  time

In [ ]:
best_ap

0.3817279399986429

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded_heuristica2_last.pth")

El AP del mejor modelo siendo esre el que guardamos es:
0.34782674967272165


In [ ]:
start_time = time.time()


for epoch in range(100, 120):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = copy.deepcopy(model.state_dict())

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.6689708232879639
Epoch: [100]  [ 0/22]  eta: 0:00:45  lr: 0.000300  time: 2.0798  data: 1.7874  max mem: 13132
1.7321898937225342
1.8185980319976807
1.6255254745483398
1.990820288658142
1.6843751668930054
1.7264271974563599
1.6774113178253174
1.7177168130874634
2.1666042804718018
1.4083960056304932
1.8127074241638184
1.6872835159301758
1.6063761711120605
1.3589080572128296
1.6770431995391846
1.5148169994354248
1.4249656200408936
1.5567762851715088
1.6932588815689087
1.7503232955932617
Epoch: [100]  [20/22]  eta: 0:00:04  lr: 0.000300  time: 2.0923  data: 1.7598  max mem: 13132
1.763777256011963
Epoch: [100]  [21/22]  eta: 0:00:02  lr: 0.000300  time: 2.0566  data: 1.7301  max mem: 13132
Epoch: [100] Total time: 0:00:45 (2.0811 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0163 (0.0163)  evaluator_time: 0.0136 (0.0136)  time: 0.0960  data: 0.0625  max mem: 13132
Test:  [39/40]  eta: 0:00:00  model_time: 0.0158 (0.0158)  evaluator_time: 0.0187 (0.

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded_heuristica2_last.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.3768483919217138


In [ ]:
start_time = time.time()


for epoch in range(120, 150):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = copy.deepcopy(model.state_dict())

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.6625475883483887
Epoch: [120]  [ 0/22]  eta: 0:00:51  lr: 0.000030  time: 2.3220  data: 1.9049  max mem: 13132
1.6206163167953491
1.6868693828582764
1.695750117301941
1.4656212329864502
1.6071211099624634
1.5572928190231323
1.4710028171539307
1.6828670501708984
1.4898377656936646
2.1025238037109375
1.6284375190734863
1.9320542812347412
1.5120762586593628
1.632815957069397
1.6582961082458496
1.4699691534042358
1.7071596384048462
2.2233033180236816
1.6755561828613281
1.5745935440063477
Epoch: [120]  [20/22]  eta: 0:00:04  lr: 0.000030  time: 2.2175  data: 1.8634  max mem: 13132
1.645188808441162
Epoch: [120]  [21/22]  eta: 0:00:02  lr: 0.000030  time: 2.2136  data: 1.8600  max mem: 13132
Epoch: [120] Total time: 0:00:48 (2.2113 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0164 (0.0164)  evaluator_time: 0.0130 (0.0130)  time: 0.0945  data: 0.0615  max mem: 13132
Test:  [39/40]  eta: 0:00:00  model_time: 0.0160 (0.0161)  evaluator_time: 0.0185 (0.0

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded_heuristica2_last.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.3817279399986429
